In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.callbacks import TensorBoard
from sklearn.externals import joblib
import datetime

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle

Using TensorFlow backend.


In [5]:
data = pd.read_csv("Test.csv", usecols=['Topic','Sentiment', 'TweetText']).append(pd.read_csv("Train.csv", usecols=['Topic','Sentiment', 'TweetText']))

In [6]:
data.Topic.value_counts()

apple        1079
twitter       953
google        867
microsoft     856
Name: Topic, dtype: int64

In [7]:
num_of_categories = 850
shuffled = data.reindex(np.random.permutation(data.index))
a = shuffled[shuffled['Topic'] == 'apple'][:num_of_categories]
t = shuffled[shuffled['Topic'] == 'twitter'][:num_of_categories]
g = shuffled[shuffled['Topic'] == 'google'][:num_of_categories]
m = shuffled[shuffled['Topic'] == 'microsoft'][:num_of_categories]
concated = pd.concat([a,t,g,m], ignore_index=True)
#Shuffle the dataset
concated = shuffle(concated)
concated['LABEL'] = 0
concated.loc[concated['Topic'] == 'apple', 'LABEL'] = 0
concated.loc[concated['Topic'] == 'twitter', 'LABEL'] = 1
concated.loc[concated['Topic'] == 'google', 'LABEL'] = 2
concated.loc[concated['Topic'] == 'microsoft', 'LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'Topic' in concated.keys():
    concated.drop(['Topic'], axis=1)
'''
 [1. 0. 0. 0.] a
 [0. 1. 0. 0.] t
 [0. 0. 1. 0.] g
 [0. 0. 0. 1.] m
'''

2244    2
826     0
1416    1
1088    1
3376    3
244     0
1451    1
2242    2
3367    3
3216    3
Name: LABEL, dtype: int64
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


'\n [1. 0. 0. 0.] a\n [0. 1. 0. 0.] t\n [0. 0. 1. 0.] g\n [0. 0. 0. 1.] m\n'

In [8]:
data.Sentiment.value_counts()

neutral       1631
irrelevant    1246
negative       475
positive       403
Name: Sentiment, dtype: int64

In [9]:
num_of_categories2 = 400
shuffled2 = data.reindex(np.random.permutation(data.index))
n = shuffled2[shuffled2['Sentiment'] == 'neutral'][:num_of_categories2]
i = shuffled2[shuffled2['Sentiment'] == 'irrelevant'][:num_of_categories2]
neg = shuffled2[shuffled2['Sentiment'] == 'negative'][:num_of_categories2]
pos = shuffled2[shuffled2['Sentiment'] == 'positive'][:num_of_categories2]
concated2 = pd.concat([n,i,neg,pos], ignore_index=True)
#Shuffle the dataset
concated2 = shuffle(concated2)
concated2['LABEL'] = 0
concated2.loc[concated2['Sentiment'] == 'neutral', 'LABEL'] = 0
concated2.loc[concated2['Sentiment'] == 'irrelevant', 'LABEL'] = 1
concated2.loc[concated2['Sentiment'] == 'negative', 'LABEL'] = 2
concated2.loc[concated2['Sentiment'] == 'positive', 'LABEL'] = 3
print(concated2['LABEL'][:10])
labels2 = to_categorical(concated2['LABEL'], num_classes=4)
print(labels2[:10])
if 'Sentiment' in concated2.keys():
    concated2.drop(['Sentiment'], axis=1)
'''
 [1. 0. 0. 0.] n
 [0. 1. 0. 0.] i
 [0. 0. 1. 0.] neg
 [0. 0. 0. 1.] pos
'''

420     1
237     0
47      0
1142    2
398     0
663     1
158     0
89      0
662     1
379     0
Name: LABEL, dtype: int64
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


'\n [1. 0. 0. 0.] n\n [0. 1. 0. 0.] i\n [0. 0. 1. 0.] neg\n [0. 0. 0. 1.] pos\n'

In [10]:
n_most_common_words = 500
max_len = 130
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['TweetText'].values)
sequences = tokenizer.texts_to_sequences(concated['TweetText'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 9338 unique tokens.


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

In [12]:
print(X_test.shape, X_train.shape)
print(y_test.shape, y_train.shape)

(850, 130) (2550, 130)
(850, 4) (2550, 4)


In [13]:
tokenizer2 = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer2.fit_on_texts(concated2['TweetText'].values)
sequences2 = tokenizer2.texts_to_sequences(concated2['TweetText'].values)
word_index2 = tokenizer2.word_index
print('Found %s unique tokens.' % len(word_index2))

X2 = pad_sequences(sequences2, maxlen=max_len)

Found 5452 unique tokens.


In [14]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2 , labels2, test_size=0.25, random_state=42)

In [15]:
print(X_test2.shape, X_train2.shape)
print(y_test2.shape, y_train2.shape)

(400, 130) (1200, 130)
(400, 4) (1200, 4)


In [16]:
def create_model(x, y, x_val, y_val, embed_dim = 128, lstm = 64, epochs = 10, batch_size = 256, optimizer='adam', verbose=1):
    model = Sequential()
    model.add(Embedding(500, embed_dim, input_length=x.shape[1]))
    model.add(SpatialDropout1D(0.7))
    model.add(LSTM(lstm, dropout=0.7, recurrent_dropout=0.7))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    print(model.summary())
    filename = "ed:{},lstm:{},ep:{},bs:{},opt:{},ts:{}".format(embed_dim, lstm, epochs, batch_size, optimizer, datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
    history = model.fit(x, y, epochs=epochs, verbose=verbose, callbacks=[TensorBoard(log_dir='tb/lstm2_' + filename, histogram_freq=0, write_graph=False)], 
                        validation_data=(x_val, y_val), batch_size=batch_size)
    #joblib.dump(model, "models/lstm_" + filename + ".sav")
    #joblib.dump(history, "models/h_" + filename + ".txt")
    return model

In [17]:
"""epochs = [10, 20, 30] #22
batch_size = [150, 250, 500] #150

embed_dims = [50, 100, 200]  #standard
lstm = [30, 50, 100]  #standard
optimizer = ['Nadam', 'Adadelta', 'Adagrad', 'Adam', 'RMSprop']  #adagrad"""

"epochs = [10, 20, 30] #22\nbatch_size = [150, 250, 500] #150\n\nembed_dims = [50, 100, 200]  #standard\nlstm = [30, 50, 100]  #standard\noptimizer = ['Nadam', 'Adadelta', 'Adagrad', 'Adam', 'RMSprop']  #adagrad"

In [18]:
cat_model = create_model(X_train, y_train, X_test, y_test, epochs=22, batch_size=150, optimizer='Adagrad')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 128)          64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 130, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 113,668
Trainable params: 113,668
Non-trainable params: 0
_________________________________________________________________
None
Train on 2550 samples, validate on 850 samples
Epoch 1/22
2550/2550 [==============================] - 3s 1ms/step - loss: 1.3160 - acc: 0.4165 - val_loss: 1.1825 - val_acc: 0.5565
Epoch 2/22
2550/2550 [==============================] - 3s 1ms/step - loss: 1.1479

In [19]:
sent_model = create_model(X_train2, y_train2, X_test2, y_test2, epochs=22, batch_size=150, optimizer='Adagrad')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 130, 128)          64000     
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 130, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
Total params: 113,668
Trainable params: 113,668
Non-trainable params: 0
_________________________________________________________________
None
Train on 1200 samples, validate on 400 samples
Epoch 1/22
1200/1200 [==============================] - 1s 1ms/step - loss: 1.3761 - acc: 0.3092 - val_loss: 1.3500 - val_acc: 0.4175
Epoch 2/22
1200/1200 [==============================] - 1s 1ms/step - loss: 1.3299

In [20]:
accr = cat_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

850/850 [==============================] - 1s 1ms/step
Test set
  Loss: 0.542
  Accuracy: 0.791


In [21]:
accr2 = sent_model.evaluate(X_test2,y_test2)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr2[0],accr2[1]))

400/400 [==============================] - 0s 1ms/step
Test set
  Loss: 0.941
  Accuracy: 0.593


In [26]:
txt = ["Getting a lot of spam at my iphone thats nice"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = cat_model.predict(padded)
labels = ['apple', 'twitter', 'google', 'microsoft']
print(pred, labels[np.argmax(pred)])
seq = tokenizer2.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = sent_model.predict(padded)
labels = ['neutral', 'irrelevant', 'negative', 'positive']
print(pred, labels[np.argmax(pred)])

[[0.61343795 0.07356836 0.15844128 0.15455246]] apple
[[0.1922596  0.00790357 0.38403144 0.41580537]] positive
